# MVP-V: 테스트 및 평가

이 노트북은 KITTI 평가 메트릭을 사용하여 모델 성능을 평가합니다.

## 목표
- KITTI 평가 메트릭 (AP, mAP 등)
- 성능 분석 및 리포트 생성

In [ ]:
# Cell 1: 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from collections import defaultdict
import json

print("평가 모듈 준비 완료")

In [ ]:
# Cell 2: IoU 계산 함수
def compute_iou(box1, box2):
    """
    두 바운딩 박스의 IoU 계산
    box: [x1, y1, x2, y2]
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    if x2 < x1 or y2 < y1:
        return 0.0
    
    inter = (x2 - x1) * (y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter
    
    return inter / union if union > 0 else 0.0

print("IoU 계산 함수 정의 완료")

In [ ]:
# Cell 3: Precision-Recall 곡선 및 AP 계산
def calculate_ap(recalls, precisions):
    """
    Average Precision (AP) 계산
    """
    # 11-point interpolation
    ap = 0.0
    for t in np.arange(0, 1.1, 0.1):
        if np.sum(recalls >= t) == 0:
            p = 0
        else:
            p = np.max(precisions[recalls >= t])
        ap += p / 11.0
    return ap

def evaluate_detections(gt_boxes, pred_boxes, iou_threshold=0.5):
    """
    탐지 결과 평가 (클래스별 AP 계산)
    
    Args:
        gt_boxes: list of [x1, y1, x2, y2, class_id]
        pred_boxes: list of [x1, y1, x2, y2, class_id, confidence]
        iou_threshold: IoU 임계값
    
    Returns:
        results: 클래스별 AP 및 전체 mAP
    """
    # 클래스별로 분리
    classes = set([box[4] for box in gt_boxes] + [box[4] for box in pred_boxes])
    results = {}
    
    for cls in classes:
        gt_cls = [box for box in gt_boxes if box[4] == cls]
        pred_cls = sorted([box for box in pred_boxes if box[4] == cls], 
                         key=lambda x: x[5], reverse=True)  # confidence로 정렬
        
        tp = np.zeros(len(pred_cls))
        fp = np.zeros(len(pred_cls))
        gt_matched = [False] * len(gt_cls)
        
        # 각 예측에 대해 매칭
        for i, pred in enumerate(pred_cls):
            best_iou = 0
            best_gt_idx = -1
            
            for j, gt in enumerate(gt_cls):
                if gt_matched[j]:
                    continue
                
                iou = compute_iou(pred[:4], gt[:4])
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = j
            
            if best_iou >= iou_threshold:
                tp[i] = 1
                gt_matched[best_gt_idx] = True
            else:
                fp[i] = 1
        
        # 누적 TP, FP
        tp_cumsum = np.cumsum(tp)
        fp_cumsum = np.cumsum(fp)
        
        # Precision, Recall
        recalls = tp_cumsum / len(gt_cls) if len(gt_cls) > 0 else np.zeros(len(pred_cls))
        precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)
        
        # AP 계산
        ap = calculate_ap(recalls, precisions)
        results[cls] = {
            'ap': ap,
            'precision': precisions,
            'recall': recalls
        }
    
    # mAP 계산
    map_score = np.mean([results[cls]['ap'] for cls in results])
    results['mAP'] = map_score
    
    return results

print("평가 함수 정의 완료")

In [ ]:
# Cell 4: KITTI 형식 평가
def evaluate_kitti_format(pred_dir, gt_dir, class_mapping=None):
    """
    KITTI 형식의 예측 및 Ground Truth 평가
    """
    if class_mapping is None:
        class_mapping = {
            'Car': 0, 'Van': 1, 'Truck': 2, 'Pedestrian': 3,
            'Person_sitting': 4, 'Cyclist': 5, 'Tram': 6, 'Misc': 7
        }
    
    pred_files = sorted(pred_dir.glob("*.txt"))
    gt_files = sorted(gt_dir.glob("*.txt"))
    
    all_results = defaultdict(list)
    
    for pred_file, gt_file in zip(pred_files, gt_files):
        # 예측 로드
        pred_boxes = []
        if pred_file.exists():
            with open(pred_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 6:
                        class_name = parts[0]
                        conf = float(parts[1])
                        x1, y1, x2, y2 = map(float, parts[2:6])
                        class_id = class_mapping.get(class_name, -1)
                        if class_id >= 0:
                            pred_boxes.append([x1, y1, x2, y2, class_id, conf])
        
        # Ground Truth 로드
        gt_boxes = []
        if gt_file.exists():
            with open(gt_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 15:
                        class_name = parts[0]
                        x1, y1, x2, y2 = map(float, parts[4:8])
                        class_id = class_mapping.get(class_name, -1)
                        if class_id >= 0:
                            gt_boxes.append([x1, y1, x2, y2, class_id])
        
        # 평가
        results = evaluate_detections(gt_boxes, pred_boxes)
        for cls, result in results.items():
            if cls != 'mAP':
                all_results[cls].append(result['ap'])
    
    # 클래스별 평균 AP
    class_aps = {}
    for cls, aps in all_results.items():
        class_aps[cls] = np.mean(aps)
    
    # 전체 mAP
    map_score = np.mean(list(class_aps.values()))
    
    return {
        'class_ap': class_aps,
        'mAP': map_score
    }

print("KITTI 평가 함수 정의 완료")

In [ ]:
# Cell 5: 결과 리포트 생성
def generate_evaluation_report(results, save_path=None):
    """
    평가 결과 리포트 생성
    """
    report = []
    report.append("=" * 60)
    report.append("KITTI 평가 결과 리포트")
    report.append("=" * 60)
    report.append("")
    
    if 'class_ap' in results:
        report.append("클래스별 AP:")
        report.append("-" * 60)
        for cls, ap in sorted(results['class_ap'].items(), key=lambda x: x[1], reverse=True):
            report.append(f"  {cls:20s}: {ap:.4f}")
        report.append("")
    
    if 'mAP' in results:
        report.append(f"mAP (mean Average Precision): {results['mAP']:.4f}")
        report.append("")
    
    report_text = "\\n".join(report)
    print(report_text)
    
    if save_path:
        with open(save_path, 'w') as f:
            f.write(report_text)
        print(f"\\n리포트 저장됨: {save_path}")
    
    return report_text

print("리포트 생성 함수 정의 완료")